In [ ]:
# cell-collective-control-kernels.ipynb
#
# Bryan Daniels
# 2023/11/2
#
# Computing control kernels for relevant networks in the cell collective database.
#

In [1]:
from neet.boolean import LogicNetwork
from neet.controlkernel.control_kernel_analysis import ck_analysis
from neet.controlkernel.modularity import pinning_produces_desired_attractor
import numpy as np
import glob
import os
from tqdm import tqdm

# Load networks from Cell Collective

In [2]:
def load_cell_collective_network(directory):
    """
    Creates neet network from cell collective data in the given directory.
    """
    expressions_file = '{}/expressions.txt'.format(directory)
    external_file = '{}/external.txt'.format(directory)
    net = LogicNetwork.read_logic(expressions_file, external_file)
    return net

In [3]:
cell_collective_directory = '../Data/Cell Collective/'

In [4]:
# load all cell collective networks into a dictionary
net_dict = {}
skip_network = 'ErbB_Receptor_Signaling' # skip this network because it is too large
for network_directory in tqdm(glob.glob(cell_collective_directory+"/*")):
    if os.path.isdir(network_directory) and not network_directory.endswith(skip_network):
        #print("Loading network from directory {}".format(network_directory))
        net_name = os.path.split(network_directory)[-1]
        net = load_cell_collective_network(network_directory)
        #print("net_name = {}".format(net_name))
        net_dict[net_name] = net

100%|███████████████████████████████████████████| 72/72 [00:07<00:00,  9.61it/s]


# Find attractors for networks with 2 attractors

In [13]:
net = net_dict['Cortical_Area_Development']
for a in net.attractors:
    print([ net.decode(state) for state in a ])

[[1, 1, 0, 0, 0]]
[[0, 0, 1, 1, 1]]


In [17]:
net = net_dict['Human_Gonadal_Sex_Determination']
for a in net.attractors:
    print([ net.decode(state) for state in a ])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1]]
[[0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0]]


# Do example control kernel analysis

In [9]:
# control kernel analysis on an example network
net = net_dict['Cortical_Area_Development']

In [64]:
# force first gene on explicitly
#net2 = load_cell_collective_network('../Data/Cell Collective/Cortical_Area_Development_forced_231102')

In [12]:
# the default code finds a single control kernel (of minimal size) for each attractor
ck_data = ck_analysis(net)
attractor_list = ck_data['attractors']
ck_list = ck_data['control_kernels']
for att,ck in zip(attractor_list,ck_list):
    ck_names = [ net.names[i] for i in ck ]
    print('Control kernel for attractor {}:'.format(np.transpose(net.decode(att))))
    print('    {}'.format(ck_names))

Finding attractors and control kernels...
Control kernel for attractor [[1 1 0 0 0]]:
    ['Emx2']
Control kernel for attractor [[0 0 1 1 1]]:
    ['Fgf8']


In [10]:
[[ net.decode(state) for state in att ] for att in net.attractors]

[[[1, 1, 0, 0, 0]], [[0, 0, 1, 1, 1]]]

In [11]:
net.basin_sizes

array([28,  4])

In [34]:
# code for finding all possible control kernels of size 1 
# (copied from Control-of-Synthetic-Boolean-Networks/code/BoolODE_data.ipynb)
def find_all_control_kernels_of_size_one(net):
    attractors = net.attractors
    for att_index in range(len(attractors)):
        desired_attractor = attractors[att_index].copy()
        desired_attractor_decoded = [net.decode(state) for state in desired_attractor]
        print("Control kernels of size one for attractor {}".format(desired_attractor_decoded))
        for i,name in enumerate(net.names):
            pin = [i,]
            pin_states = [[desired_attractor_decoded[0][i],],]
            if pinning_produces_desired_attractor(net,pin,pin_states,desired_attractor):
                print({name})

In [59]:
for i in range(5):
    print("g{} <-> {}".format(i+1,net.names[i]))

g1 <-> Coup_fti
g2 <-> Emx2
g3 <-> Fgf8
g4 <-> Sp8
g5 <-> Pax6


In [56]:
find_all_control_kernels_of_size_one(net)

Control kernels of size one for attractor [[1, 1, 0, 0, 0]]
{'Emx2'}
{'Fgf8'}
{'Sp8'}
Control kernels of size one for attractor [[0, 0, 1, 1, 1]]
{'Fgf8'}


### What are the Cortical Area Network's dynamics from some example initial conditions?

In [104]:
def simulate_to_attractor(net,state):
    traj = []
    while state not in traj:
        traj.append(state.copy())
        state = net.update(traj[-1].copy())
    return traj

In [109]:
# starting from the initial conditions in AY's "pRogress_4"
[ print(state) for state in simulate_to_attractor(net,[0,0,1,1,0]) ];

[0, 0, 1, 1, 0]
[0, 0, 1, 1, 1]


In [110]:
[ print(state) for state in simulate_to_attractor(net,[0,0,0,1,0]) ];

[0, 0, 0, 1, 0]
[0, 0, 0, 0, 1]
[1, 0, 0, 0, 0]
[1, 1, 0, 0, 0]


In [111]:
[ print(state) for state in simulate_to_attractor(net,[0,0,0,0,0]) ];

[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 1, 0, 0, 0]


In [112]:
[ print(state) for state in simulate_to_attractor(net,[1,0,1,1,0]) ];

[1, 0, 1, 1, 0]
[0, 0, 1, 1, 0]
[0, 0, 1, 1, 1]


# Find Cell Collective networks that have only two attractors 

In [50]:
two_attractor_names = []
for network_name in net_dict.keys():
    print("Finding attractors for {}...".format(network_name))
    net = net_dict[network_name]
    try:
        print("    Number of attractors = {}".format(len(net.attractors)))
        if len(net.attractors) == 2:
            two_attractor_names.append(network_name)
    except ValueError:
        print("    Error")    

Finding attractors for Arabidopsis_Thaliana_Cell_Cycle...
    Number of attractors = 1
Finding attractors for T-LGL_Survival_Network_2008...
    Error
Finding attractors for Wg_Pathway_Of_Drosophila_Signalling_Pathways...


KeyboardInterrupt: 